In [3]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

INPUT_FILE  = r"/Users/yinhaiyi/Desktop/工作簿4.xlsx"   # 原始长表
SHEET_NAME  = "Sheet1"
OUTPUT_FILE = r"/Users/yinhaiyi/Desktop/all_companies_wide.xlsx"

CUSTOM_ORDER = ["Symbol", "EndDate", "ShortName", "销售人员", "技术人员", "财务人员", "合计"]

def process_one_company(symbol_code: str,
                        input_file=INPUT_FILE,
                        sheet_name=SHEET_NAME,
                        output_file=OUTPUT_FILE,
                        replace_if_exists=True):
    """处理单个公司并把宽表追加到总Excel（sheet=公司代码）"""
    sym = str(symbol_code).strip().zfill(6)

    # 读取数据
    df = pd.read_excel(input_file, sheet_name=sheet_name)

    # 清理 Symbol
    df["Symbol"] = (df["Symbol"].astype(str)
                                .str.replace(r"\.0$", "", regex=True)
                                .str.replace(r"[^\d]", "", regex=True)
                                .str.zfill(6))

    # 去除空格
    for col in ["EmployDetail", "ShortName", "EmployStructure"]:
        df[col] = df[col].astype(str).str.strip()

    # 合并重复键
    df = (df.groupby(["Symbol", "EndDate", "ShortName", "EmployStructure", "EmployDetail"],
                     as_index=False)["Amount"].sum())

    # 只取该公司
    sub = df[df["Symbol"] == sym].copy()
    if sub.empty:
        raise SystemExit(f"未找到公司 {sym} 的数据")

    # === 给列加前缀 ===
    is_edu = sub["EmployStructure"].str.contains("学", na=False)
    sub["DetailPrefixed"] = np.where(is_edu,
                                     "edu_" + sub["EmployDetail"],
                                     "job_" + sub["EmployDetail"])

    # 长转宽
    wide = (sub.pivot_table(index=["Symbol", "EndDate", "ShortName"],
                            columns="DetailPrefixed",
                            values="Amount",
                            aggfunc="sum")
               .reset_index())
    wide.columns.name = None

    # 保存
    if os.path.exists(output_file):
        from openpyxl import load_workbook
        wb = load_workbook(output_file)
        if sym in wb.sheetnames and replace_if_exists:
            wb.remove(wb[sym])
            wb.save(output_file)
        with pd.ExcelWriter(output_file, engine="openpyxl", mode="a", if_sheet_exists="new") as w:
            wide.to_excel(w, sheet_name=sym, index=False)
    else:
        with pd.ExcelWriter(output_file, engine="xlsxwriter") as w:
            wide.to_excel(w, sheet_name=sym, index=False)

    print(f"✅ 已写入：{output_file}（sheet: {sym}）")

# 测试
process_one_company("300866")

✅ 已写入：/Users/yinhaiyi/Desktop/all_companies_wide.xlsx（sheet: 300866）


In [4]:
file = r"/Users/yinhaiyi/Desktop/all_companies_wide.xlsx"

xls = pd.ExcelFile(file)
print(xls.sheet_names)

df = pd.read_excel(file, sheet_name="300866")
print(df.head())

['002095', '002102', '002127', '002152', '002264', '002315', '002499', '002640', '002803', '002959', '003010', '300209', '300226', '300464', '300518', '300592', '300740', '300783', '300785', '300792', '300840', '300892', '300998', '301001', '301110', '301381', '301558', '600539', '601113', '603613', '603808', '605136', '300866']
   Symbol     EndDate ShortName  edu_合计  edu_大专及以下  edu_大学本科  edu_本科  \
0  300866  2020-12-31      安克创新    2144        289       NaN  1447.0   
1  300866  2021-12-31      安克创新    3532        401    2341.0     NaN   
2  300866  2023-12-31      安克创新    4017        344       NaN  2738.0   
3  300866  2024-12-31      安克创新    5034        159       NaN  3762.0   

   edu_硕士及以上  job_合计  job_客服人员  job_市场人员  job_技术人员  job_生产人员  job_研发人员  \
0        408    2144     155.0     160.0       NaN       NaN    1010.0   
1        790    3532     286.0     350.0       NaN       NaN    1605.0   
2        935    4017       NaN       NaN    1918.0       0.0       NaN   
3       1113